# Analysis of H3K27ac ChIP-seq in T cells from dCas9-p300 mice  

### Notes

1) 20/12/07 - Issue with contaminating reads from ATAC-seq in CRE+g5-rep3 H3K27ac ChIP-seq from running on the same flow cell. 
   - We wrote code below to remove the extra reads
   
2) 20/12/07 - Peak caller was not picking up the FoxP3 region; therefore merged replicates and re-ran MACS2 broad peak caller to use for union set
  

In [ ]:
#Code from for demultiplexing contaminating reads
%%bash
comm  -2 -3  \
    <(gzip -dc /data/reddylab/Keith/collab/200924_Gemberling/data/chip_seq/processed_raw_reads/Siklenka_6683_201201A5/KS162_CrePos_g5_rep3.R1.fastq.gz  \
        | awk 'NR%4==1{res=$1}NR%4>1{res=res"\t"$1}NR%4==0{res=res"\t"$1; print res}'  \
        | sort -k1,1) \
    <(gzip -dc /data/reddylab/Keith/encode4_duke/data/atac_seq/processed_raw_reads/Keith_6683_201201A5/KS136_Th17ASTARRInput_rep1.R1.fastq.gz \
        | awk 'NR%4==1{res=$1}NR%4>1{res=res"\t"$1}NR%4==0{res=res"\t"$1; print res}' \
        | sort -k1,1) \
| tr '\t' '\n' \
| gzip -c > /data/reddylab/Alex/tmp/cleaned.KS162_CrePos_g5_rep3.R1.fastq.gz
comm  -2 -3  \
    <(gzip -dc /data/reddylab/Keith/collab/200924_Gemberling/data/chip_seq/processed_raw_reads/Siklenka_6683_201201A5/KS162_CrePos_g5_rep3.R2.fastq.gz  \
        | awk 'NR%4==1{res=$1}NR%4>1{res=res"\t"$1}NR%4==0{res=res"\t"$1; print res}'  \
        | sort -k1,1) \
    <(gzip -dc /data/reddylab/Keith/encode4_duke/data/atac_seq/processed_raw_reads/Keith_6683_201201A5/KS136_Th17ASTARRInput_rep1.R2.fastq.gz \
        | awk 'NR%4==1{res=$1}NR%4>1{res=res"\t"$1}NR%4==0{res=res"\t"$1; print res}' \
        | sort -k1,1) \
| tr '\t' '\n' \
| gzip -c > /data/reddylab/Alex/tmp/cleaned.KS162_CrePos_g5_rep3.R2.fastq.gz

In [ ]:
# # Convert bed file to SAF format for featureCount input
## 20/12/08 - take q3 of broadPeak; convert to saf for feature counts
# %%bash
source /data/reddylab/software/miniconda2/bin/activate alex
OUTDIR=/gpfs/fs1/data/reddylab/Keith/collab/200924_Gemberling/analysis/
python /data/reddylab/Alex/reddylab_utils/scripts/bed_to_saf.py \
    --bed \
-beds /gpfs/fs1/data/reddylab/Keith/collab/200924_Gemberling/analysis/201209_Gemberling2012_K27acChIP-broadPeak-UnionPeakset-filtq3.bed \
-safs ${OUTDIR}/201209_Gemberling2012_K27acChIP-broadPeak-UnionPeaksetFiltq3.saf


In [ ]:
#ChIP-seq K27ac featureCounts from broadPeaks
#Generate count table for k27ac ChIP-seq

###Generate count table with featureCounts in paired end mode
%%bash
cd /gpfs/fs1/data/reddylab/Keith/collab/200924_Gemberling/analysis
sbatch -pnew,all \
    --cpus-per-task 16 \
    --mem 32G \
    -o /gpfs/fs1/data/reddylab/Keith/collab/200924_Gemberling/analysis/logs/201209_Gemberling2012_K27acChIP.broadPeakFiltq3.featureCounts.out \
    <<'EOF'
#!/bin/bash
/data/reddylab/software/subread-1.4.6-p4-Linux-x86_64/bin/featureCounts \
    -T 16 \
    -F SAF \
    -p \
-a /gpfs/fs1/data/reddylab/Keith/collab/200924_Gemberling/analysis/201209_Gemberling2012_K27acChIP-broadPeak-UnionPeaksetFiltq3.saf \
-o /gpfs/fs1/data/reddylab/Keith/collab/200924_Gemberling/analysis/201209_Gemberling2012_K27acChIP.broadPeakFiltq3.featureCounts.txt \
/gpfs/fs1/data/reddylab/Keith/collab/200924_Gemberling/processing/chip_seq/Siklenka_6621_201109A5-pe/KS157.CREneg.g5.K27ac.rep1.masked.dedup.sorted.bam \
/gpfs/fs1/data/reddylab/Keith/collab/200924_Gemberling/processing/chip_seq/Siklenka_6621_201109A5-pe/KS157.CREneg.g5.K27ac.rep2.masked.dedup.sorted.bam \
/gpfs/fs1/data/reddylab/Keith/collab/200924_Gemberling/processing/chip_seq/Siklenka_6621_201109A5-pe/KS157.CREneg.g5.K27ac.rep3.masked.dedup.sorted.bam \
/gpfs/fs1/data/reddylab/Keith/collab/200924_Gemberling/processing/chip_seq/Siklenka_6683_201201A5-pe/KS162_CrePos_g5_rep1.masked.dedup.sorted.bam \
/gpfs/fs1/data/reddylab/Keith/collab/200924_Gemberling/processing/chip_seq/Siklenka_6683_201201A5-pe/KS162_CrePos_g5_rep2.masked.dedup.sorted.bam \
/gpfs/fs1/data/reddylab/Keith/collab/200924_Gemberling/processing/chip_seq/Siklenka_6683_201201A5-pe/cleaned.comm.KS162_CrePos_g5_rep3.masked.dedup.sorted.bam \
/gpfs/fs1/data/reddylab/Keith/collab/200924_Gemberling/processing/chip_seq/Siklenka_6683_201201A5-pe/KS162_CrePos_NTC_rep1.masked.dedup.sorted.bam \
/gpfs/fs1/data/reddylab/Keith/collab/200924_Gemberling/processing/chip_seq/Siklenka_6683_201201A5-pe/KS162_CrePos_NTC_rep2.masked.dedup.sorted.bam \
/gpfs/fs1/data/reddylab/Keith/collab/200924_Gemberling/processing/chip_seq/Siklenka_6683_201201A5-pe/KS162_CrePos_NTC_rep3.masked.dedup.sorted.bam \

EOF

In [2]:
##Load featureCounts File
colnm <- c("Geneid","Chr","Start","End","Strand","Length",
"KS157_CREneg_g5.rep1",
"KS157_CREneg_g5.rep2",
"KS157_CREneg_g5.rep3",
"KS162_CrePos_g5.rep1",
"KS162_CrePos_g5.rep2",
"KS162_CrePos_g5.rep3",
"KS162_CrePos_NTC.rep1",
"KS162_CrePos_NTC.rep2",
"KS162_CrePos_NTC.rep3"
           )

bpFeatureCounts <- read.table("/gpfs/fs1/data/reddylab/Keith/collab/200924_Gemberling/analysis/201209_Gemberling2012_K27acChIP.broadPeakFiltq3.featureCounts.txt", 
                              header=TRUE, 
                              col.names = colnm)

head(bpFeatureCounts)

,Geneid,Chr,Start,End,Strand,Length,KS157_CREneg_g5.rep1,KS157_CREneg_g5.rep2,KS157_CREneg_g5.rep3,KS162_CrePos_g5.rep1,KS162_CrePos_g5.rep2,KS162_CrePos_g5.rep3,KS162_CrePos_NTC.rep1,KS162_CrePos_NTC.rep2,KS162_CrePos_NTC.rep3
,<fct>,<fct>,<int>,<int>,<fct>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>
1,GL456216.1_15807_17235,GL456216.1,15807,17235,+,1429,529,548,339,132,84,175,224,260,203
2,GL456216.1_49148_49467,GL456216.1,49148,49467,+,320,20,14,10,11,5,23,31,18,30
3,GL456233.1_38782_39154,GL456233.1,38782,39154,+,373,32,33,16,19,6,19,16,30,20
4,GL456233.1_159559_160218,GL456233.1,159559,160218,+,660,48,44,30,27,17,36,42,50,26
5,GL456239.1_12211_13278,GL456239.1,12211,13278,+,1068,37,37,21,43,21,61,51,64,57
6,GL456239.1_22604_22884,GL456239.1,22604,22884,+,281,7,11,10,26,10,37,33,26,29


In [ ]:
#Filter out nonstandard and mitochondrial chromosomes
AllChrs <- unique(bpFeatureCounts[,2])
inxStd <- grep(x = unique(bpFeatureCounts[,2]), pattern="chr[0-9, X, Y]")
inxSelect <- bpFeatureCounts[,2] %in% AllChrs[inxStd] 
bpFeatureCounts <- bpFeatureCounts[inxSelect,]

#get coordinates for Foxp3 guide targeted peak
inxFox <- grep("chrX_7578787_7580637", bpFeatureCounts[,1])
bpFeatureCounts[inxFox,]

#### Deseq2 comparing dCas9-p300::Cre+ treated with targeting Foxp3 or non targeting gRNA

In [3]:
#Set up colData
cts <- bpFeatureCounts[,-c(1:6)]
colData <- as.matrix(colnames(cts))
genoType <- as.matrix(c(rep("CREneg",3),rep("CREpos", 6)))
guideType <- as.matrix(c(rep("g5", 6), rep("NTC", 3)))
treatmentType <- as.matrix(c(rep("control",3),rep("treat", 3), rep("control",3)))

colData <- cbind(colData, genoType, guideType, treatmentType)
colnames(colData) <- c("sampleID", "genoType", "guideType", "treatmentType")
colData

sampleID,genoType,guideType,treatmentType
KS157_CREneg_g5.rep1,CREneg,g5,control
KS157_CREneg_g5.rep2,CREneg,g5,control
KS157_CREneg_g5.rep3,CREneg,g5,control
KS162_CrePos_g5.rep1,CREpos,g5,treat
KS162_CrePos_g5.rep2,CREpos,g5,treat
KS162_CrePos_g5.rep3,CREpos,g5,treat
KS162_CrePos_NTC.rep1,CREpos,NTC,control
KS162_CrePos_NTC.rep2,CREpos,NTC,control
KS162_CrePos_NTC.rep3,CREpos,NTC,control


In [ ]:
require(DESeq2)
dds <- DESeqDataSetFromMatrix(countData=cts[,-c(1:3)],
                             colData=colData[-c(1:3),],
                             design=~guideType)


dds$guideType <- relevel(dds$guideType, "NTC")
resultsNames(dds)

dds <- DESeq(dds)

res <- results(dds, independentFiltering=FALSE)
summary(res, alpha = 0.01)

#Write the results object; unfiltered
setwd("/gpfs/fs1/data/reddylab/Keith/collab/200924_Gemberling/analysis")
out <- cbind(bpFeatureCounts[,1:6], res)
# write.table(x = out, file="201211_Gemberling2020_K27acChIP-seq_DESeq2_CREpos_G5_v_NTC_allpeaks.tsv", quote = FALSE, row.names = FALSE, sep="\t")

In [5]:
#Write table of filtered hits padj≤0.01
inxSig <- res$padj <= 0.01
inxFC <- abs(res$log2FoldChange) >= 0

hits <- res[inxFC & inxSig,]
hit.peaks <- bpFeatureCounts[inxFC & inxSig,1:6]
o <- order(hits$log2FoldChange, decreasing = TRUE)
hitsBed <- cbind(hit.peaks[o,],hits[o,])
hitsBed
# write.table(x = hitsBed, file="201209_Gemberling2020_K27acChIP-seq_DESeq2_hits_q2.tsv", quote = FALSE, row.names = FALSE, sep="\t")

ERROR: Error in eval(expr, envir, enclos): object 'res' not found


#### Plot MA for Supplementary Figure 5B

In [4]:
#201209 - PlotMA from q3 cutoff, all 3 replicates, standard peaks
# pdf(file = "./plots/201010_Gemberling2020_G5vNTC_CRE+_q3_MAplot.pdf", useDingbats=FALSE)
plotMA(res, ylim=c(-4,4), las=1, alpha=0.01, cex=0.75)
text(x = res[inxFox,1], y=res[inxFox,2]+0.25, "Foxp3 gRNA-5")
points(x=res[inxFox,1], y=res[inxFox,2], cex=1.2, col="orange")
# dev.off()

ERROR: Error in plotMA(res, ylim = c(-4, 4), las = 1, alpha = 0.01, cex = 0.75): could not find function "plotMA"


#### Deseq2 comparison for Foxp3 gRNA treated Cre+ vs Cre- H3K27ac ChIP-seq

In [ ]:
require(DESeq2)
dds <- DESeqDataSetFromMatrix(countData=cts[,-c(7:9)],
                             colData=colData[-c(7:9),],
                             design=~genoType)


dds$guideType <- relevel(dds$genoType, "CREneg")
resultsNames(dds)

dds <- DESeq(dds)

res <- results(dds, independentFiltering=FALSE)
summary(res, alpha = 0.01)

#### Plot MA for Supplementary Figure 5E

In [ ]:
#201209 - PlotMA from q3 cutoff, all 3 replicates, standard peaks

#get coordinates for Foxp3 guide targeted peak
inxFox <- grep("chrX_7578787_7580637", bpFeatureCounts[,1])
bpFeatureCounts[inxFox,]

# pdf(file = "./plots/201010_Gemberling2020_CREposVCREneg_g5_q3_MAplot.pdf", useDingbats=FALSE)
plotMA(res, ylim=c(-4,4), las=1, alpha=0.01, cex=0.75)
text(x = res[inxFox,1], y=res[inxFox,2]+0.25, "Foxp3 gRNA-5")
points(x=res[inxFox,1], y=res[inxFox,2], cex=1.2, col="orange")
# dev.off()

In [ ]:
#Filter hits on q2 but no effect size
inxSig <- which(res$padj <= 0.01)
# inxFC <- abs(res$log2FoldChange) >= 0

hits <- res[inxSig,]
hit.peaks <- bpFeatureCounts[inxSig,1:6]
o <- order(hits$log2FoldChange, decreasing = TRUE)
hitsBed <- cbind(hit.peaks[o,],hits[o,])
hitsBed
write.table(x = hitsBed, file="201209_Gemberling2020_K27acChIP-seq_DESeq2_hits_CREpos_v_CREneg_q2.tsv", quote = FALSE, row.names = FALSE, sep="\t")



### Figure 4G
#### Make CPM bar plots at FoxP3 locus

In [ ]:
#Get counts from featureCounts table
inxFox <- grep("chrX_7578787_7580637", bpFeatureCounts[,1])

cpms <- cpm(cts)
fox.cpms <- as.numeric(cpms[inxFox,,drop=FALSE])
fox.table <- cbind(colData, cpm=melt(fox.cpms)$value) #melt table for ggplot

fox.df <- as.data.frame(fox.table, stringsAsFactors=FALSE)
fox.df$cpm <- as.numeric(fox.df$cpm)
interact <- interaction(fox.df$guideType, fox.df$genoType, drop = TRUE)
level_order <- c("g5.CREneg", "NTC.CREpos", "g5.CREpos")
interact <- factor(interact, levels = level_order)
fox.df$interact <- interact

fox.df

#Summarize means
(df_cpm_means <- fox.df %>%
  group_by(interact) %>%   # the grouping variable
  summarise(mean_cpm = mean(cpm), sd = sd(cpm)))fox.df <- as.data.frame(fox.table)


In [6]:
#Plot
set.seed(31339)
g <- ggplot(fox.df, aes(x=interact, y=cpm, group=interact))
g2 <-  g + geom_boxplot(data=fox.df, notch=FALSE, width=0.5) +
    stat_boxplot(geom = "errorbar", width=0.25) +
    geom_jitter(aes(shape=interact), size=3, width=0.075) +
    scale_x_discrete(expand=c(0.8,0)) +
    scale_y_continuous(expand=c(0.3,0), breaks=c(0, 25, 50, 75, 100, 125), limits=c(0,125)) +
    scale_shape_manual(values=c(16, 15, 17)) +
	labs(y="CPM") + 
	theme(panel.grid.major = element_blank(), panel.grid.minor = element_blank(),
panel.background = element_blank(), axis.line = element_line(colour = "black"))

g2

ERROR: Error in ggplot(fox.df, aes(x = interact, y = cpm, group = interact)): could not find function "ggplot"


# Analysis of flow cytometry data from T cells treated with single gRNA targeting the Foxp3 promoter 
### Supplemental Figure2D-E

In [ ]:
library(ggplot2)
library(gridExtra)
library(gtools)
library(dplyr)
flowDatFile <- "~/Documents/Computing/Collaboration/210420_Gemberling_NMETH/201212_KS149_AllgRNAFlow_d7.txt"

dat <- read.table(file=flowDatFile, header=TRUE, sep="\t")

level_order <- mixedsort(levels(dat[,4]), decreasing=FALSE)

 #Plot FoxP3

(df_means <- dat %>%
  group_by(guideType, genoType) %>%   # the grouping variable
  summarise(mean_freq = mean(Foxp3.freq), sd = sd(Foxp3.freq)))


# pdf(file="./Plots/201212_Gemberling2020_SupplementaryFigure_singlegRNAScreen_barPlot.pdf", useDingbats=FALSE)
g <- ggplot(df_means, aes(x=factor(guideType, level = level_order), y=mean_freq))
g2 <- g + geom_col(data=df_means, fill="NA", color="black") +
	scale_y_continuous(limits=c(0,100)) +
	labs(title = "% of FoxP3+ Lymphocytes after 7d", x="gRNA", y="%FOXP3-eGFP+ Cells") +
	theme(axis.text.x=element_text(angle=90, hjust=0.95)) + 
	geom_point(data=dat, aes(x=factor(guideType, level = level_order), y=Foxp3.freq)) +
	theme(panel.grid.major = element_blank(), panel.grid.minor = element_blank(),
panel.background = element_blank(), axis.line = element_line(colour = "black")) +
	# geom_errorbar(aes(ymin=mean_freq-sd, ymax=mean_freq+sd, width=0.2)) +
	facet_grid(genoType ~ .)
g2
# dev.off()


### Statistical testing for Supplementary Figure 2D-E


In [ ]:
library(multcomp)
library(agricolae)

flowDatFile <- "~/Documents/Computing/Collaboration/210420_Gemberling_NMETH/201212_KS149_AllgRNAFlow_d7.txt"
dat <- read.table(file=flowDatFile, header=TRUE, sep="\t")


#Setup data
dat_grp <- dat %>% group_by(guideType, genoType)

rmvTreg <- dat_grp$guideType %in% "Treg" 
rmvTh0 <- dat_grp$guideType %in% "Th0" 
inxKRAB <- dat_grp$genoType %in% "KRAB"
inxp300 <- dat_grp$genoType %in% "p300"


#Remove Th0 and Treg positive controls for the ANOVA and split into KRAB and p300
dat_krab <- dat_grp[inxKRAB & !rmvTh0 & !rmvTreg,]
dat_p300 <- dat_grp[inxp300 & !rmvTh0 & !rmvTreg,]

levels(dat_krab$guideType)



In [ ]:
#Run ANOVA for KRAB

#Change reference to NTC
dat_krab$guideType <- relevel(dat_krab$guideType, ref = "NTC")
model2 <- aov(Foxp3.freq ~ guideType, data=dat_krab)
summary(model2)
post_test <- glht(model2, linfct=mcp(guideType = "Dunnett"))
summary(post_test)

In [ ]:
#Run ANOVA for p300 

#Change reference to NTC
dat_p300$guideType <- relevel(dat_p300$guideType, ref = "NTC")
model1 <- aov(Foxp3.freq ~ guideType, data=dat_p300)
summary(model1)
post_test <- glht(model1, linfct=mcp(guideType = "Dunnett"))
summary(post_test)